In [52]:
import warnings
import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests

warnings.filterwarnings('ignore')


# bringing in url
nwis_url = (
    "https://waterdata.usgs.gov/nwis"
    "/dv?cb_00060=on&format=rdb&site_no="
    "11141280&legacy=&referred_module=sw&"
    "period=&begin_date=1941-09-30&end_date"
    "=2023-08-31"
)

# Send an HTTP GET request to the URL
nwis_response = requests.get(nwis_url)
nwis_response.raise_for_status()




In [53]:
# import data
slo_q_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment='#',
    delimiter='\t',
    skiprows=[29,30],
    names=['agency_cd', 'site_no', 'datetime', 'streamflow_cfs', 'code'],
    usecols=[i for i in range(5)],
    index_col='datetime',
    parse_dates=True,
    na_values = 'Ice'
)


In [54]:
slo_ann_max_q_df = slo_q_df[['streamflow_cfs']].resample('AS').max()

In [55]:
slo_ann_max_q_post_df = slo_ann_max_q_df["1967":].copy()

slo_ann_max_q_post_df["exeedance_prob"] = slo_ann_max_q_post_df.rank(
    ascending=False
) / len(slo_ann_max_q_post_df)
slo_ann_max_q_post_df["return_period"] = (
    1 / slo_ann_max_q_post_df.exeedance_prob
)

In [ ]:
slo_ann_max_q_post_df.return_period.hvplot(title = 'Flood Return Period in SLO, CA from 1967-2023', 
    xlabel = 'Month/Year (hourly)', ylabel = 'Return Period (Years)')

:Curve   [datetime]   (return_period)

In [58]:
%%capture
%%bash
jupyter nbconvert time-series-data-SLO.ipynb --to html --no-input

# Despite San Luis Obispo, California flooding this year, the largest flood occured in 1969.

In 2023, much of San Luis Obispo, California flooded due to higher than unsual rains (Img. 1). Despite the extremity of the flood, this is not the most intense flooding San Luis Obispo has seen. In 1969, the maximum recorded streamflow in San Luis Obispo was 1,360 cubic feet per second which is nearly 3 times as much as the maximum recorded streamflow in 2023 (447 cfs). More information on San Luis Obispo floods can be found here: (https://www.ksby.com/homepage-showcase/a-look-back-at-major-flooding-in-san-luis-obispo-county#:~:text=City%20records%20and%20photos%20from,inches%20in%20half%20an%20hour.%E2%80%9D)

### Data Citation
U.S. Geological Survey, 2016, National Water Information System data available on the World Wide Web (USGS Water Data for the Nation), accessed [Sept 13, 2023], at URL ["https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=11141280&legacy=&referred_module=sw&period=&begin_date=1941-09-30&end_date=2023-08-31"]